In [24]:
import numpy as np
import copy

Parameters and function setting

In [25]:
def solution(x, y):
    return np.sin(3.0 * np.pi * x ** 2 * y) + (y - 1.0) ** 3

In [26]:
def f(x, y):
    return 18 * np.pi * (np.pi * x ** 2 * np.sin(3 * np.pi * x ** 2 * y) * (x ** 2 + 6 * y ** 2) - y * np.cos(3 * np.pi * x ** 2 * y)) - 12 * (y - 1)

In [27]:
def a(x, y):
    return 3

def b(x, y):
    return 2

SLAE compilation

In [28]:
def SLAE_compilation(N: int) -> tuple[np.ndarray, np.ndarray]:
    def idx(i, j):
        return (j - 1) * (N - 1) + (i - 1)

    n = (N - 1) ** 2

    A = np.zeros((n, n))
    b_vec = np.zeros(n)

    h = 1.0 / N

    for j in range(1, N):  # y
        for i in range(1, N):  # x
            k = idx(i, j)
            A[k, k] = -2 * (a(0, 0) + b(0, 0)) / h**2
            x, y = i * h, j * h
            b_k = -f(x, y)

            for di, dj, coeff in [(-1, 0, a), (1, 0, a), (0, -1, b), (0, 1, b)]:
                ni, nj = i + di, j + dj
                if 1 <= ni <= N - 1 and 1 <= nj <= N - 1:
                    A[k, idx(ni, nj)] = coeff(0, 0) / h**2
                else:
                    xg, yg = ni * h, nj * h
                    b_k += coeff(0, 0) * solution(xg, yg) / h**2

            b_vec[k] = b_k

    return A, b_vec

Conjugate Gradient Method

In [53]:
def conjugate_gradient(A: np.ndarray, b_vec: np.ndarray, eps: float = 1e-2) -> np.ndarray:
    x_cur = np.zeros_like(b_vec)
    r_cur = b_vec - A @ x_cur
    p_cur = copy.deepcopy(r_cur)

    while np.linalg.norm(r_cur) > eps:
        r_prev = copy.deepcopy(r_cur)
        p_prev = copy.deepcopy(p_cur)
        alpha_prev = np.dot(r_prev, r_prev) / np.dot(A @ p_prev, p_prev)
        x_prev = copy.deepcopy(x_cur)
        x_cur = x_prev + alpha_prev * p_prev
        r_cur = r_prev - alpha_prev * A @ p_prev
        beta_prev = np.dot(r_cur, r_cur) / np.dot(r_prev, r_prev)
        p_cur = r_cur + beta_prev * p_prev
        print(f"norm(r_cur) = {np.linalg.norm(r_cur)}\n")
    return x_cur

In [ ]:
N = 4
A, b_vec = SLAE_compilation(N)

x = conjugate_gradient(A=A, b_vec=b_vec)

x